In [ ]:
"""
CÓDIGO PARA GENERAR EL MODELO DE PREDICCIÓN DE USOS DEL SUELO DESDE EL ESPACIO

"""


#librerías necesarias 
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  print('o')
import tensorflow as tf


from tensorflow.keras.models import load_model
print(tf.__version__)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
"""
DEFINICIÓN DE LAS VARIABLES NECESARIAS PARA NUESTRA CNN:

"""
#tamaño de imagen
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_SHAPE = (IMG_HEIGHT,IMG_WIDTH,3)

#Tamaño del lote de imágenes
batch_size = 32
lr = 0.0001 #tasa de aprendizaje para el entrenamiento de la red
epochs = 150

In [ ]:
"""
CARGAR LOS DATOS DE ENTRENAMIENTO Y TEST

"""

#rutas necesarias
path = "../input/usos-del-suelo-desde-el-espacio/"
train_path = path + "train/train/"
test_path = path + "validacion/"

In [ ]:
#Cargamos los datos, usando data augmentation
#Usaremos un 20% de los datos para validar nuestro modelo
train_datagen = ImageDataGenerator(rescale=1/255,
                                   zoom_range=0.5,
                                   rotation_range=45,
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1/255)

train_generator= train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_path, # same directory as training data
    target_size=(IMG_HEIGHT,IMG_WIDTH),
    batch_size=batch_size,
    subset='validation') # set as validation data

test_generator= test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

In [ ]:
"""
DEFINIR EL MODELO. USAREMOS TRANSFER LEARNING PARTIENDO DE XCEPTION

"""
#Si lo ejecutamos en kaggle asegurarnos de que Settings > Internet esta en On para descargarlo
from tensorflow.keras.applications.xception import Xception

xception = Xception(
    input_shape=IMG_SHAPE,
    include_top=False, #definimos nuestra entrada para imagenes con IMG_SHAPE
    weights="imagenet" #cargamos los pesos
)

#No vamos a congelar ninguna capa, vamos a entrenar todos los pesos
#for layer in inception_v3.layers:
#    layer.trainable=False
    
xception.summary()

In [ ]:
#Añadimos capas propias a Xception

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization, Conv2D

model = tf.keras.models.Sequential([
    xception,
    
    Flatten(),
    Dense(512,activation="relu"),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(256,activation="relu"),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(45, activation='softmax')
])

model.summary()

In [ ]:
#Compilamos el modelo que hemos creado

# Base learning rate
base_learning_rate = lr
# Compile the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [ ]:
#Entrenamos el modelo
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

history = model.fit(
    train_generator,
    epochs=epochs,
    verbose=1,
    callbacks=[callback],
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data = validation_generator
)

In [ ]:
"""
PREDECIR SOBRE LOS DATOS DE TEST
"""
preds = model.predict(test_generator, verbose = 1)

In [ ]:
"""
GUARDAR EN CSV LOS RESULTADOS
"""

import glob,os
import pandas as pd
import numpy as np

filename = "../working/" + "output-96-v2.csv"
filename2 = "../working/" + "output_con_prob-96.csv"

ids = [f[11:] for f in test_generator.filenames]

solution = pd.DataFrame(data = {"ID": ids, "Category": np.argmax(preds, axis = 1)})
solution.to_csv(filename, index = False, columns = ["ID", "Category"])

solution2 = pd.DataFrame(data = {"ID": ids, "Category": np.argmax(preds, axis = 1), "Pred":  np.amax(preds, axis = 1)})
solution2.to_csv(filename2, index = False, columns = ["ID", "Category","Pred"])

In [ ]:
"""
GUARDAR EL MODELO
"""
model.save("../working/modeloXception-96-v2.h5")